In [0]:
from pathlib import Path
import os
import os, json, sys
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, Iterable, List
def resolve_root() -> Path:
    candidates: List[Path] = []
    if os.environ.get("MAIN_ROOT"):
        candidates.append(Path(os.environ["MAIN_ROOT"]))
    if getattr(sys, "argv", None) and sys.argv and sys.argv[0]:
        candidates.append(Path(sys.argv[0]).resolve().parent)
    candidates.append(Path(os.getcwd()))
    for c in candidates:
        for base in [c, *c.parents]:
            m = base / "main"
            if (m / "documents").exists():
                return m
    return Path(os.getcwd()) / "main"

In [0]:
root = resolve_root()
doc_root = root / "documents"
out_file = root / "out" / "chunks_unified.jsonl"
print(root)
print(doc_root)
print(out_file)

In [0]:
##/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/out/chunks_unified.jsonl

In [0]:

df = spark.read.json(
    "file:/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/out/Nano Family/Nano 33 BLE/chunks.jsonl"
)
display(df)


In [0]:
df_2 = spark.read.json(
    "file:/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/out/Nano Family/Nano 33 BLE/chunks_sample.jsonl"
)
display(df_2)

In [0]:
ROOT = get_project_root()

DOC_ROOT = ROOT / "documents"
OUT_DIR  = ROOT / "out"
OUT_FILE = OUT_DIR / "chunks_level0.jsonl"

OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"ROOT     = {ROOT}")
print(f"DOC_ROOT = {DOC_ROOT}")
print(f"OUT_DIR  = {OUT_DIR}")
print(f"OUT_FILE = {OUT_FILE}")

In [0]:
/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/chunking/docling/chunker_docling_hybrid.py

In [0]:
from pathlib import Path
import os

root = Path(os.getcwd())       # kein __file__, kein Ärger
doc_root = root / "documents"
out_dir = root / "out"

print(root)
print(doc_root)
print(out_dir)

In [0]:
!pip install docling


In [0]:
dbutils.fs.ls(
    "file:/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/documents/Nano Family/Nano 33 BLE/"
)

In [0]:


# ✅ Neuer offizieller Weg
from docling.document_converter import DocumentConverter

# Pfad zur Datei (PDF oder HTML)
pdf_path_ = "/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/documents/Nano Family/Nano 33 BLE/Nano_33_BLE_datasheet.pdf"
# 1️⃣ Converter erzeugen
converter = DocumentConverter()

# 2️⃣ Datei konvertieren (Docling erkennt Typ automatisch)
result = converter.convert(pdf_path_)

# 3️⃣ Zugriff auf das Docling-Dokument
doc = result.document

# 4️⃣ Ausgabe




In [0]:
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
pdf_options = PdfPipelineOptions()
pdf_options.do_ocr = False                    # No OCR - pure text extraction only   # noqa: E501
pdf_options.generate_page_images = False      # No page images  # noqa: E501
pdf_options.generate_picture_images = False   # Ignore pictures completely  # noqa: E501
pdf_options.generate_table_images = False     # Keep tables as text/markdown, not images  # noqa: E501

        # Configure format options
format_options = {
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pdf_options
            )
        }

        # Initialize document converter
converter_2 = DocumentConverter(
            format_options=format_options
        )
result_2 = converter_2.convert(pdf_path_)
doc_2 = result_2.document

In [0]:
print(doc_2.pages)

In [0]:
print(doc.pages)

In [0]:
print(doc_2.export_to_markdown()[:1000])

In [0]:
%pip install -qU pip docling transformers

In [0]:
DOC_SOURCE = doc
from docling.chunking import HybridChunker

chunker = HybridChunker()
chunk_iter = chunker.chunk(dl_doc=doc)

In [0]:
for i, chunk in enumerate(chunk_iter):
    print(f"=== {i} ===")
    print(f"chunk.text:\n{f'{chunk.text[:300]}…'!r}")

    enriched_text = chunker.contextualize(chunk=chunk)
    print(f"chunker.contextualize(chunk):\n{f'{enriched_text[:300]}…'!r}")

    print()

In [0]:
from docling_core.transforms.chunker.tokenizer.huggingface import HuggingFaceTokenizer
from transformers import AutoTokenizer

from docling.chunking import HybridChunker

EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
MAX_TOKENS = 500  # set to a small number for illustrative purposes

tokenizer = HuggingFaceTokenizer(
    tokenizer=AutoTokenizer.from_pretrained(EMBED_MODEL_ID),
    max_tokens=MAX_TOKENS,  # optional, by default derived from `tokenizer` for HF case
)
chunker = HybridChunker(
    tokenizer=tokenizer,
    merge_peers=True,  # optional, defaults to True
)
chunk_iter = chunker.chunk(dl_doc=doc)
chunks = list(chunk_iter)
chunks

In [0]:
import re

def clean_text(t: str) -> str:
    if not t:
        return ""

    # 1) Zeilenumbrüche & Silbentrennung
    #    „micro-\ncontroller“ -> „microcontroller“
    t = re.sub(r"(\w)-\n(\w)", r"\1\2", t)
    t = t.replace("\r", "")
    t = re.sub(r"\n{2,}", "\n", t)             # Mehrfach-Blankzeilen -> eine
    t = re.sub(r"[ \t]{2,}", " ", t)           # Mehrfach-Spaces -> einer

    # 2) Kopf-/Fußzeilen & Seitenzahlen (heuristisch)
    #    Zeilen, die fast nur aus Ziffern/Sonderzeichen bestehen, entfernen
    def noisy(line: str) -> bool:
        s = line.strip()
        if not s:
            return True
        non_alpha = sum(1 for ch in s if not ch.isalpha())
        return (non_alpha / max(1, len(s))) > 0.6

    lines = [ln for ln in t.split("\n") if not noisy(ln)]
    t = "\n".join(lines)

    # 3) Tabellen-/Figure-Captions & Tabellenraster raus
    t = re.sub(r"^(Table|Figure)\s*\d+[:.\-]\s.*$", "", t, flags=re.IGNORECASE | re.MULTILINE)
    t = re.sub(r"^\s*\|.*\|\s*$", "", t, flags=re.MULTILINE)  # Markdown-Tabellenreihen
    t = re.sub(r"^\s*[-=]{3,}\s*$", "", t, flags=re.MULTILINE)

    # 4) Kurze Reste weg
    t = t.strip()
    return t

# Schritt 1: Chunking
chunk_iter = chunker.chunk(dl_doc=doc)
chunks = list(chunk_iter)   # jetzt hast du eine Liste von Chunk-Objekten

# Schritt 2: Iteration über rohe Chunks
for i, ch in enumerate(chunks):
    print(f"=== Chunk {i} ===")

    # Text reinigen
    raw = clean_text(ch.text or "")
    if len(raw) < 30:
        continue

    # Kontextualisierter Text
    ctx = clean_text(chunker.contextualize(chunk=ch))

    # Token zählen
    txt_tokens = tokenizer.count_tokens(raw)
    ctx_tokens = tokenizer.count_tokens(ctx)

    print(f"Raw ({txt_tokens} tokens): {raw[:500]!r}\n")
    print(f"Context ({ctx_tokens} tokens): {ctx[:500]!r}")
    print("=" * 80)


In [0]:
import json
from pathlib import Path

def clean_text(s: str) -> str:
    """Einfache Bereinigung (wie zuvor besprochen)."""
    if not s:
        return ""
    import re
    s = re.sub(r"(\w)-\n(\w)", r"\1\2", s)     # Silbentrennung
    s = s.replace("\r", "").strip()
    s = re.sub(r"\n+", " ", s)
    return s

# Ausgabe-Ordner bestimmen
source_path= Path("/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/documents/Nano Family/Nano 33 BLE/Nano_33_BLE_datasheet.pdf")
out_dir = Path("/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/out/Nano Family/Nano 33 BLE")
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "chunks.jsonl"

with open(out_path, "w", encoding="utf-8") as f:
    for i, ch in enumerate(chunks):
        raw = clean_text(ch.text or "")
        if len(raw) < 30:  # zu kurze ignorieren
            continue
        ctx = clean_text(chunker.contextualize(chunk=ch))

        meta = {
            "source": str(source_path),
            "chunk_index": i,
            "page_span": getattr(ch, "page_span", None),
        }

        row = {
            "id": f"{Path(source_path).stem}::c{i}",
            "text_raw": raw,
            "text_ctx": ctx,
            "meta": meta,
        }

        f.write(json.dumps(row, ensure_ascii=False) + "\n")

print(f"[OK] {out_path} geschrieben ✅")


In [0]:
df = spark.read.json ("file:/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/out/Nano Family/Nano 33 BLE/chunks.jsonl")
df.display()

In [0]:
for i, chunk in enumerate(chunks):
    print(f"=== {i} ===")
    txt_tokens = tokenizer.count_tokens(chunk.text)
    print(f"chunk.text ({txt_tokens} tokens):\n{chunk.text!r}")

    ser_txt = chunker.contextualize(chunk=chunk)
    ser_tokens = tokenizer.count_tokens(ser_txt)
    print(f"chunker.contextualize(chunk) ({ser_tokens} tokens):\n{ser_txt!r}")

    print()

In [0]:
# Databricks-Notebook Zelle
!pip install readability-lxml lxml beautifulsoup4 --quiet


In [0]:
from pathlib import Path

def resolve_main_root() -> Path:
    """
    Sucht im aktuellen Arbeitsverzeichnis (und Eltern) nach einem Ordner 'main'
    mit einem Unterordner 'documents'. Funktioniert robust im Notebook.
    """
    cwd = Path.cwd()
    for base in [cwd, *cwd.parents]:
        m = base / "main"
        if (m / "documents").exists():
            return m
    raise RuntimeError("Konnte 'main/documents' nicht finden – bitte Notebook im Repo/Workspace öffnen.")

main_root = resolve_main_root()
doc_root = main_root / "documents"
doc_root


In [0]:
# Beispiel: wähle exakt deine beiden Dateien
#/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/documents/Nano Family/Nano 33 BLE/Controlling RGB LED Through Bluetooth® _ Arduino Documentation.html
# /Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/documents/Nano Family/Nano 33 BLE/Connecting Two Nano 33 BLE Boards Through I2C _ Arduino Documentation.html
html_paths = [
    doc_root / "Nano Family" / "Nano 33 BLE" / "Controlling RGB LED Through Bluetooth® _ Arduino Documentation.html",
    doc_root / "Nano Family" / "Nano 33 BLE" / "Connecting Two Nano 33 BLE Boards Through I2C _ Arduino Documentation.html",
]

# Optional: falls du einfach die ersten 2-3 Tutorials automatisch nehmen willst:
# html_paths = list(doc_root.rglob("*/tutorials/*.html"))[:3]

# Existenz prüfen:
for p in html_paths:
    print(p, "EXISTS" if p.exists() else "MISSING")


In [0]:
from readability import Document
from bs4 import BeautifulSoup

def clean_html_with_readability(path: Path):
    raw = path.read_text(encoding="utf-8", errors="ignore")
    doc = Document(raw)
    title = doc.short_title() or path.stem
    content_html = doc.summary()  # gereinigtes HTML
    # Zusätzlich Plaintext extrahieren (optional, praktisch zum Debuggen)
    text = BeautifulSoup(content_html, "html.parser").get_text("\n", strip=True)
    return {
        "path": str(path),
        "title": title,
        "clean_html": content_html,
        "clean_text": text
    }

cleaned = [clean_html_with_readability(p) for p in html_paths if p.exists()]
len(cleaned), [c["title"] for c in cleaned]


In [0]:
# HTML-Vorschau (nacheinander)
for item in cleaned:
    print("=== TITLE:", item["title"])
    displayHTML(item["clean_html"])  # Databricks rendert den bereinigten Inhalt


In [0]:
# Erste 800 Zeichen Plaintext je Datei ausgeben
for item in cleaned:
    print("\n" + "="*80)
    print("TITLE:", item["title"])
    print("PATH :", item["path"])
    print("-"*80)
    preview = item["clean_text"][:800]
    print(preview + ("..." if len(item["clean_text"]) > 800 else ""))


In [0]:
%pip install trafilatura --quiet


In [0]:
import trafilatura
from pathlib import Path

def clean_html_with_trafilatura(path: Path):
    raw = path.read_text(encoding="utf-8", errors="ignore")
    downloaded = trafilatura.extract(raw, include_comments=False, include_tables=True, include_links=False)
    return {
        "path": str(path),
        "title": path.stem,
        "clean_text": downloaded or "",
    }


In [0]:
cleaned_traf = [clean_html_with_trafilatura(p) for p in html_paths if p.exists()]
print(len(cleaned_traf), "files processed")

for c in cleaned_traf:
    print("\n" + "="*80)
    print("FILE:", c["path"])
    print("EXTRACTED TEXT PREVIEW:\n")
    print(c["clean_text"][:800] + "..." if len(c["clean_text"]) > 800 else c["clean_text"])


In [0]:
import pandas as pd
df = pd.DataFrame([{"file": c["path"], "chars": len(c["clean_text"])} for c in cleaned_traf])
display(df)


In [0]:
from pathlib import Path

def peek(path: Path, n=1200):
    raw = path.read_text(encoding="utf-8", errors="ignore")
    print(raw[:n])
p1 = Path("/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/documents/Nano Family/Nano 33 BLE/Controlling RGB LED Through Bluetooth® _ Arduino Documentation.html")
peek(p1)



In [0]:
# In Databricks (oder lokal):
# Falls schon installiert, kannst du die Zeile überspringen.
%pip install --quiet langchain-text-splitters langchain-core beautifulsoup4 pymupdf


In [0]:
INPUT_ROOT = Path("/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/documents")  # Wurzel mit Kategorien/Produkten
OUTPUT_ROOT = Path("/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/out")       # Hierhin schreiben wir JSONL

# Teste zuerst EIN Produkt (step-by-step).
TEST_CATEGORY = "Nano Family"   # z.B. "Nano"
TEST_PRODUCT  = "Nano 33 BLE"    # z.B. "A000005"

# Chunk-Parameter (gute Startwerte für technische Texte)
CHUNK_SIZE = 800
CHUNK_OVERLAP = 120
INPUT_ROOT

In [0]:
import re
from typing import List, Tuple
from bs4 import BeautifulSoup
import fitz  # PyMuPDF

def clean_text_post(text: str) -> str:
    """
    Post-Cleaning:
    - Markdown-Tabellen-Separatoren und Pipe-Blöcke weglassen (heuristisch),
    - Linien-/Deko-Zeilen entfernen,
    - Mehrfach-Leerzeilen reduzieren,
    - Trim.
    """
    cleaned_lines = []
    skip_table_block = False
    for line in text.splitlines():
        l = line.strip()
        # Trennlinie/Separator einer Markdown-Tabelle?
        if re.match(r"^\|?(\s*:?-{3,}:?\s*\|)+\s*$", l):
            skip_table_block = True
            continue
        if skip_table_block:
            if not l or "|" not in l:
                skip_table_block = False
            continue
        # Deko-Linien (-----, =====, _____)
        if re.match(r"^[\-_=\s]{5,}$", l):
            continue
        cleaned_lines.append(line)

    text = "\n".join(cleaned_lines)
    text = re.sub(r"\n\s*\n\s*\n+", "\n\n", text).strip()
    return text
def read_pdf_clean(pdf_path: Path) -> str:
    """
    Extrahiert reinen Text aus PDF (ohne Bilder).
    Hinweis: Tabellen in PDFs sind oft als Text 'gezeichnet' → heuristische Nachreinigung.
    """
    parts = []
    try:
        with fitz.open(pdf_path) as doc:
            for page in doc:
                parts.append(page.get_text("text"))
    except Exception as e:
        print(f"[WARN] PDF konnte nicht gelesen werden: {pdf_path} -> {e}")
        return ""
    return clean_text_post("\n".join(parts))
    
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

SECTION_PATTERNS = [
    r'^#{1,6}\s+(.+)$',        # Markdown-Header
    r'^.+\n[=\-]{2,}\s*$',     # unterstrichene Header
    r'^[A-Z][A-Z\s0-9\-_/]+:$' # ALL CAPS mit Doppelpunkt
]
STOPWORDS = set(['the','and','is','of','to','a','in','that','it','with','as','for','on','this','by','an','be'])


In [0]:
def perform_semantic_chunking(document_text: str, chunk_size: int, chunk_overlap: int) -> List[Document]:
    splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ". ", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    chunks = splitter.split_text(document_text)

    docs: List[Document] = []
    current_section = "Introduction"

    for i, chunk in enumerate(chunks):
        for line in chunk.splitlines():
            line = line.strip()
            for pat in SECTION_PATTERNS:
                m = re.match(pat, line, flags=re.MULTILINE)
                if m:
                    current_section = m.group(1) if m.groups() else m.group(0)
                    break

        words = re.findall(r'\b\w+\b', chunk.lower())
        density = 0.0 if not words else round(
            len([w for w in words if w not in STOPWORDS]) / max(1, len(words)), 3
        )

        docs.append(
            Document(
                page_content=chunk,
                metadata={
                    "chunk_id": i,
                    "total_chunks": len(chunks),
                    "chunk_size": len(chunk),
                    "chunk_type": "semantic",
                    "section": current_section,
                    "semantic_density": density
                }
            )
        )
    return docs


In [0]:
def read_product_texts(category: str, product: str) -> List[Tuple[str, str]]:
    """
    Liest alle relevanten Dateien eines Produkts und gibt Liste (source_file_name, cleaned_text) zurück.
    Erwartete Struktur:
      main/documents/<category>/<product>/{*.pdf, tutorial_*.html}
    """
    base = INPUT_ROOT / category / product
    results: List[Tuple[str, str]] = []

    # PDFs
    for pdf in sorted(base.glob("*.pdf")):
        t = read_pdf_clean(pdf)
        if t.strip():
            results.append((pdf.name, t))



    return results

read_product_texts(category=TEST_CATEGORY, product=TEST_PRODUCT) 

read_product_texts

In [0]:
def make_master_text(sources: List[Tuple[str, str]]) -> str:
    """
    Kombiniert mehrere Quelldateien in einen Text mit klaren SOURCE-Markern.
    Der Splitter findet dadurch natürliche Grenzen besser.
    """
    parts = []
    for name, txt in sources:
        parts.append(f"## SOURCE: {name}\n\n{txt.strip()}")
    return "\n\n---\n\n".join(parts)
make_master_text

In [0]:
import json

def write_jsonl(records, out_path: Path):
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with out_path.open("w", encoding="utf-8") as f:
        for rec in records:
            f.write(json.dumps(rec, ensure_ascii=False) + "\n")

def process_one_product(category: str, product: str, chunk_size: int, chunk_overlap: int) -> int:
    sources = read_product_texts(category, product)
    if not sources:
        print(f"[INFO] Keine Quellen unter {INPUT_ROOT / category / product}")
        return 0

    master_text = make_master_text(sources)
    docs = perform_semantic_chunking(master_text, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    # JSONL-Records
    records = []
    for d in docs:
        records.append({
            "category": category,
            "product": product,
            "chunk_id": f"{category}:{product}:{d.metadata['chunk_id']:04d}",
            "total_chunks": d.metadata["total_chunks"],
            "chunk_type": d.metadata["chunk_type"],
            "section": d.metadata["section"],
            "semantic_density": d.metadata["semantic_density"],
            "chunk_size": d.metadata["chunk_size"],
            "text": d.page_content
        })

    out_path = OUTPUT_ROOT / category / product / "chunks_sample.jsonl"
    write_jsonl(records, out_path)
    print(f"[OK] {category}/{product}: {len(records)} Chunks → {out_path}")
    return len(records)


In [0]:
_ = process_one_product(
    TEST_CATEGORY,
    TEST_PRODUCT,
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)


In [0]:
import json, itertools

out_file = OUTPUT_ROOT / TEST_CATEGORY / TEST_PRODUCT / "chunks_sample.jsonl"

df = spark.read.json("file:/Workspace/Users/nasiba.tuychieva@gea.com/ai_samples/main/out/Nano Family/Nano 33 BLE/chunks_sample.jsonl")
display(df)

In [0]:
from pathlib import Path

# ➜ Passe diese beiden Pfade an DEIN Projekt an:
INPUT_ROOT = Path("main/documents")  # Wurzel mit Kategorien/Produkten
OUTPUT_ROOT = Path("main/out")       # Hierhin schreiben wir JSONL

# Teste zuerst EIN Produkt (step-by-step).
TEST_CATEGORY = "your_category"   # z.B. "Nano"
TEST_PRODUCT  = "your_product"    # z.B. "A000005"

# Chunk-Parameter (gute Startwerte für technische Texte)
CHUNK_SIZE = 800
CHUNK_OVERLAP = 120


In [0]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import re

def perform_semantic_chunking(document, chunk_size=500, chunk_overlap=100:disappointed_face:
    """
    Performs semantic chunking on a document using recursive character splitting 
    at logical text boundaries.
    
    Args:
        document (str): The text document to process
        chunk_size (int): The target size of each chunk in characters
        chunk_overlap (int): The number of characters of overlap between chunks
        
    Returns:
        list: The semantically chunked documents with metadata
    """
    # Create the text splitter with semantic separators
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ". ", " ", ""],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    
    # Split the text into semantic chunks
    semantic_chunks = text_splitter.split_text(document)
    print(f"Document split into {len(semantic_chunks)} semantic chunks")
    
    # Determine section titles for enhanced metadata
    section_patterns = [
        r'^#+\s+(.+)$',      # Markdown headers
        r'^.+\n[=\-]{2,}$',  # Underlined headers
        r'^[A-Z\s]+:$'       # ALL CAPS section titles
    ]
    
    # Convert to Document objects with enhanced metadata
    documents = []
    current_section = "Introduction"
    
    for i, chunk in enumerate(semantic_chunks):
        # Try to identify section title from chunk
        chunk_lines = chunk.split('\n')
        for line in chunk_lines:
            for pattern in section_patterns:
                match = re.match(pattern, line, re.MULTILINE)
                if match:
                    current_section = match.group(0)
                    break
        
        # Calculate semantic density (ratio of non-stopwords to total words)
        words = re.findall(r'\b\w+\b', chunk.lower())
        stopwords = ['the', 'and', 'is', 'of', 'to', 'a', 'in', 'that', 'it', 'with', 'as', 'for']
        content_words = [w for w in words if w not in stopwords]
        semantic_density = len(content_words) / max(1, len(words))
        
        doc = Document(
            page_content=chunk,
            metadata={
                "chunk_id": i,
                "total_chunks": len(semantic_chunks),
                "chunk_size": len(chunk),
                "chunk_type": "semantic",
                "section": current_section,
                "semantic_density": round(semantic_density, 2)
            }
        )
        documents.append(doc)
    
    return documents